In [1]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#=======================8万=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'479'}), frozenset({'681'}), frozenset({'170'}), frozenset({'2505'}), frozenset({'13032'}), frozenset({'3294'}), frozenset({'201'}), frozenset({'9579'}), frozenset({'338'}), frozenset({'548'}), frozenset({'110'}), frozenset({'2046'}), frozenset({'12929'}), frozenset({'14099'}), frozenset({'1046'}), frozenset({'136'}), frozenset({'789'}), frozenset({'766'}), frozenset({'589'}), frozenset({'4424'}), frozenset({'1313'}), frozenset({'2150'}), frozenset({'76'}), frozenset({'39', '48'}), frozenset({'32', '48'}), frozenset({'48', '19'}), frozenset({'101', '48'}), frozenset({'185', '48'}), frozenset({'161', '39'}), frozenset({'1327', '48'}), frozenset({'48', '89'}), frozenset({'39', '237'}), frozenset({'38', '48'}), frozenset({'39', '3270'}), frozenset({'39', '1146'}), frozenset({'39', '225'}), frozenset({'522', '39'}), frozenset({'39', '1004'}), frozenset({'39', '49'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'48', '78'}), frozenset({'39', '186'}), frozenset({'179', '48'}), frozenset({'237', '32'}), frozenset({'38', '225'}), frozenset({'32', '225'}), frozenset({'39', '740'}), frozenset({'31', '39'}), frozenset({'56', '38'}), frozenset({'264', '48'}), frozenset({'2958', '48'}), frozenset({'39', '156'}), frozenset({'10', '39'}), frozenset({'38', '237'}), frozenset({'48', '425'}), frozenset({'592', '39', '48'}), frozenset({'301', '39', '48'}), frozenset({'310', '39', '48'}), frozenset({'286', '39', '38'}), frozenset({'39', '258', '48'}), frozenset({'39', '48', '809'}), frozenset({'475', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'270', '271', '48'}), frozenset({'39', '48', '413'}), frozenset({'2706'}), frozenset({'6343'}), frozenset({'251'}), frozenset({'1543'}), frozenset({'379'}), frozenset({'13443'}), frozenset({'15033'}), frozenset({'1481'}), frozenset({'703'}), frozenset({'677'}), frozenset({'374'}), frozenset({'3149'}), frozenset({'2343'}), frozenset({'3904'}), frozenset({'14855'}), frozenset({'1486'}), frozenset({'3539'}), frozenset({'1859'}), frozenset({'1966'}), frozenset({'647'}), frozenset({'53'}), frozenset({'1198'}), frozenset({'2259'}), frozenset({'433'}), frozenset({'514'}), frozenset({'739'}), frozenset({'178'}), frozenset({'38', '1146'}), frozenset({'23', '39'}), frozenset({'39', '1121'}), frozenset({'39', '1986'}), frozenset({'438', '41'}), frozenset({'396', '48'}), frozenset({'32', '1146'}), frozenset({'271', '41'}), frozenset({'107', '39'}), frozenset({'2056', '39'}), frozenset({'1344', '41'}), frozenset({'123', '41'}), frozenset({'48', '769'}), frozenset({'48', '675'}), frozenset({'1593', '48'}), frozenset({'48', '175'}), frozenset({'200', '48'}), frozenset({'39', '9617'}), frozenset({'855', '39', '48'}), frozenset({'170', '48', '41'}), frozenset({'110', '48', '41'}), frozenset({'533', '39', '41'}), frozenset({'270', '39', '41'}), frozenset({'39', '41', '117'}), frozenset({'39', '1034', '769'}), frozenset({'39', '38', '170', '41'}), frozenset({'39', '38', '110', '41'}), frozenset({'270', '39', '48', '41'}), frozenset({'10605'}), frozenset({'643'}), frozenset({'412'}), frozenset({'2492'}), frozenset({'650'}), frozenset({'38', '389'}), frozenset({'39', '389'}), frozenset({'389', '32'}), frozenset({'10446', '48'}), frozenset({'39', '12491'}), frozenset({'12473', '48'}), frozenset({'48', '12503'}), frozenset({'56', '39'}), frozenset({'39', '2241'}), frozenset({'1404', '48'}), frozenset({'10515', '39', '48'}), frozenset({'39', '1578', '48'}), frozenset({'39', '48', '649'}), frozenset({'286', '38', '39', '48'}), frozenset({'2097'}), frozenset({'652'}), frozenset({'15578'}), frozenset({'15832', '41'}), frozenset({'10423', '48'}), frozenset({'12925', '41'}), frozenset({'13556', '39'}), frozenset({'1435', '48'}), frozenset({'150', '48'}

In [2]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#=======================分区：4=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=4
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'592', '39', '48'}), frozenset({'301', '39', '48'}), frozenset({'310', '39', '48'}), frozenset({'286', '39', '38'}), frozenset({'475', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'10515', '39', '48'}), frozenset({'270', '271', '48'}), frozenset({'39', '48', '413'}), frozenset({'286', '39', '38', '48'}), frozenset({'110'}), frozenset({'1903'}), frozenset({'338'}), frozenset({'2046'}), frozenset({'12929'}), frozenset({'2310'}), frozenset({'548'}), frozenset({'53'}), frozenset({'14099'}), frozenset({'15618'}), frozenset({'15659'}), frozenset({'13443'}), frozenset({'251'}), frozenset({'766'}), frozenset({'789'}), frozenset({'589'}), frozenset({'136'}), frozenset({'412'}), frozenset({'10605'}), frozenset({'1198'}), frozenset({'379'}), frozenset({'677'}), frozenset({'703'}), frozenset({'2492'}), frozenset({'170'}), frozenset({'211'}), frozenset({'178'}), frozenset({'681'}), frozenset({'76'}), frozenset({'201'}), frozenset({'479'}), frozenset({'2364'}), frozenset({'374'}), frozenset({'237', '32'}), frozenset({'39', '48'}), frozenset({'38', '48'}), frozenset({'32', '48'}), frozenset({'38', '237'}), frozenset({'39', '237'}), frozenset({'48', '89'}), frozenset({'48', '19'}), frozenset({'179', '48'}), frozenset({'39', '1004'}), frozenset({'48', '497'}), frozenset({'438', '41'}), frozenset({'2958', '48'}), frozenset({'32', '225'}), frozenset({'10446', '48'}), frozenset({'161', '39'}), frozenset({'10', '39'}), frozenset({'39', '3270'}), frozenset({'264', '48'}), frozenset({'56', '38'}), frozenset({'39', '2241'}), frozenset({'185', '48'}), frozenset({'39', '488'}), frozenset({'39', '225'}), frozenset({'1327', '48'}), frozenset({'39', '1146'}), frozenset({'39', '156'}), frozenset({'48', '78'}), frozenset({'101', '48'}), frozenset({'23', '39'}), frozenset({'38', '225'}), frozenset({'31', '39'}), frozenset({'271', '41'}), frozenset({'48', '175'}), frozenset({'39', '806'}), frozenset({'39', '389'}), frozenset({'38', '389'}), frozenset({'389', '32'}), frozenset({'39', '740'}), frozenset({'107', '39'}), frozenset({'39', '49'}), frozenset({'2343'}), frozenset({'1543'}), frozenset({'1486'}), frozenset({'1313'}), frozenset({'2150'}), frozenset({'2626'}), frozenset({'2505'}), frozenset({'13032'}), frozenset({'3149'}), frozenset({'9579'}), frozenset({'522', '39'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'39', '186'}), frozenset({'12951', '39'}), frozenset({'48', '425'}), frozenset({'39', '258', '48'}), frozenset({'39', '48', '649'}), frozenset({'270', '39', '41'}), frozenset({'110', '48', '41'}), frozenset({'170', '48', '41'}), frozenset({'38', '39', '170', '41'}), frozenset({'38', '39', '110', '41'}), frozenset({'39', '41', '533'}), frozenset({'310', '32', '48'}), frozenset({'855', '39', '48'}), frozenset({'39', '117', '41'}), frozenset({'39', '48', '809'}), frozenset({'13041', '39', '32'}), frozenset({'39', '38', '13041'}), frozenset({'39', '13041', '175'}), frozenset({'15618', '48', '41'}), frozenset({'16010', '32', '48'}), frozenset({'16010', '39', '48'}), frozenset({'16010', '38', '48'}), frozenset({'39', '16011', '41'}), frozenset({'39', '32', '16217'}), frozenset({'39', '38', '16217'}), frozenset({'39', '16431', '48'}), frozenset({'48', '39', '16011', '41'}), frozenset({'1046'}), frozenset({'514'}), frozenset({'15879'}), frozenset({'15578'}), frozenset({'10515', '405'}), frozenset({'48', '267'}), frozenset({'1435', '48'}), frozenset({'150', '48'}), frozenset({'39', '80'}), frozenset({'39', '1986'}), frozenset({'39', '12943'}), frozenset({'12925', '41'}), frozenset({'13041', '413'}), frozenset({'41', '15832'}), frozenset({'10423', '48'}), frozenset({'348'}), frozenset({'1013'}), frozenset({'1233'}), frozenset({'433'}), frozenset({'1859'}), frozenset({'3966'

In [3]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#=======================分区：8=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'589'}), frozenset({'789'}), frozenset({'1313'}), frozenset({'514'}), frozenset({'178'}), frozenset({'76'}), frozenset({'2626'}), frozenset({'1591'}), frozenset({'32', '48'}), frozenset({'39', '225'}), frozenset({'39', '48'}), frozenset({'32', '225'}), frozenset({'1435', '48'}), frozenset({'48', '267'}), frozenset({'39', '237'}), frozenset({'264', '48'}), frozenset({'23', '39'}), frozenset({'48', '78'}), frozenset({'39', '1146'}), frozenset({'209', '39'}), frozenset({'39', '806'}), frozenset({'38', '48'}), frozenset({'38', '237'}), frozenset({'185', '48'}), frozenset({'2958', '48'}), frozenset({'39', '49'}), frozenset({'522', '39'}), frozenset({'48', '89'}), frozenset({'48', '19'}), frozenset({'38', '225'}), frozenset({'56', '38'}), frozenset({'179', '48'}), frozenset({'101', '48'}), frozenset({'161', '39'}), frozenset({'12951', '39'}), frozenset({'39', '740'}), frozenset({'1158', '39'}), frozenset({'39', '156'}), frozenset({'48', '425'}), frozenset({'107', '39'}), frozenset({'39', '3270'}), frozenset({'237', '32'}), frozenset({'39', '1004'}), frozenset({'2988', '39'}), frozenset({'39', '12943'}), frozenset({'808', '48'}), frozenset({'10', '39'}), frozenset({'48', '175'}), frozenset({'39', '13043'}), frozenset({'310', '39', '48'}), frozenset({'39', '48', '413'}), frozenset({'475', '39', '48'}), frozenset({'301', '39', '48'}), frozenset({'286', '38', '39'}), frozenset({'592', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'418', '39', '48'}), frozenset({'10515', '39', '48'}), frozenset({'286', '38', '39', '48'}), frozenset({'13443'}), frozenset({'251'}), frozenset({'1543'}), frozenset({'703'}), frozenset({'677'}), frozenset({'1481'}), frozenset({'201'}), frozenset({'170'}), frozenset({'479'}), frozenset({'13032'}), frozenset({'681'}), frozenset({'110'}), frozenset({'338'}), frozenset({'2046'}), frozenset({'12929'}), frozenset({'548'}), frozenset({'53'}), frozenset({'14099'}), frozenset({'15618'}), frozenset({'766'}), frozenset({'136'}), frozenset({'1046'}), frozenset({'433'}), frozenset({'15578'}), frozenset({'10515', '405'}), frozenset({'31', '39'}), frozenset({'150', '48'}), frozenset({'39', '80'}), frozenset({'1239', '48'}), frozenset({'396', '48'}), frozenset({'39', '1986'}), frozenset({'39', '186'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'39', '9617'}), frozenset({'12925', '41'}), frozenset({'13041', '413'}), frozenset({'271', '41'}), frozenset({'41', '15832'}), frozenset({'65', '15832'}), frozenset({'10423', '48'}), frozenset({'13334', '39', '48'}), frozenset({'39', '258', '48'}), frozenset({'39', '5152', '48'}), frozenset({'32', '48', '413'}), frozenset({'38', '48', '413'}), frozenset({'39', '48', '649'}), frozenset({'39', '48', '809'}), frozenset({'39', '13041', '89'}), frozenset({'13041', '39', '32'}), frozenset({'39', '38', '13041'}), frozenset({'39', '13041', '175'}), frozenset({'32', '39', '13041', '48'}), frozenset({'39', '38', '170', '41'}), frozenset({'39', '38', '13041', '48'}), frozenset({'48', '39', '13041', '175'}), frozenset({'39', '13041', '48', '89'}), frozenset({'379'}), frozenset({'15879'}), frozenset({'2505'}), frozenset({'374'}), frozenset({'3294'}), frozenset({'1903'}), frozenset({'2310'}), frozenset({'15659'}), frozenset({'15'}), frozenset({'13060'}), frozenset({'438', '41'}), frozenset({'13556', '39'}), frozenset({'249', '41'}), frozenset({'39', '1715'}), frozenset({'39', '2673'}), frozenset({'14098', '16010'}), frozenset({'413', '16217'}), frozenset({'809', '16217'}), frozenset({'170', '48', '41'}), frozenset({'39', '533', '41'}), frozenset({'270', '39', '41'}), frozenset({'15618', '48', '41'}), frozenset({'110', '48', '41'}), frozenset({'2184', '39', '41'}), frozenset({'39', '41', '14933'}), frozenset({'12929',

In [4]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#=======================分区：12=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=12
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'53'}), frozenset({'348'}), frozenset({'840'}), frozenset({'2343'}), frozenset({'381'}), frozenset({'589'}), frozenset({'766'}), frozenset({'789'}), frozenset({'1233'}), frozenset({'677'}), frozenset({'3074'}), frozenset({'76'}), frozenset({'178'}), frozenset({'201'}), frozenset({'2505'}), frozenset({'110'}), frozenset({'338'}), frozenset({'1013'}), frozenset({'1198'}), frozenset({'2046'}), frozenset({'2413'}), frozenset({'251'}), frozenset({'1096'}), frozenset({'703'}), frozenset({'1481'}), frozenset({'3966'}), frozenset({'170'}), frozenset({'479'}), frozenset({'681'}), frozenset({'4198'}), frozenset({'548'}), frozenset({'4636'}), frozenset({'1859'}), frozenset({'2492'}), frozenset({'374'}), frozenset({'3840'}), frozenset({'48', '89'}), frozenset({'179', '48'}), frozenset({'189', '48'}), frozenset({'39', '237'}), frozenset({'39', '225'}), frozenset({'38', '225'}), frozenset({'19', '48'}), frozenset({'1327', '48'}), frozenset({'39', '1004'}), frozenset({'48', '749'}), frozenset({'1344', '41'}), frozenset({'39', '48'}), frozenset({'38', '48'}), frozenset({'39', '49'}), frozenset({'39', '94'}), frozenset({'32', '48'}), frozenset({'249', '41'}), frozenset({'161', '39'}), frozenset({'48', '352'}), frozenset({'39', '740'}), frozenset({'522', '39'}), frozenset({'31', '39'}), frozenset({'39', '1121'}), frozenset({'39', '1146'}), frozenset({'32', '1146'}), frozenset({'185', '48'}), frozenset({'2958', '48'}), frozenset({'48', '2990'}), frozenset({'101', '48'}), frozenset({'38', '237'}), frozenset({'48', '78'}), frozenset({'237', '32'}), frozenset({'23', '39'}), frozenset({'271', '41'}), frozenset({'438', '41'}), frozenset({'32', '1715'}), frozenset({'39', '1715'}), frozenset({'32', '225'}), frozenset({'39', '3270'}), frozenset({'286', '38', '39'}), frozenset({'475', '39', '48'}), frozenset({'270', '39', '41'}), frozenset({'170', '48', '41'}), frozenset({'39', '32', '604'}), frozenset({'310', '32', '48'}), frozenset({'592', '39', '48'}), frozenset({'110', '48', '41'}), frozenset({'301', '39', '48'}), frozenset({'1859', '48', '41'}), frozenset({'39', '533', '41'}), frozenset({'310', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'39', '38', '170', '41'}), frozenset({'39', '38', '110', '41'}), frozenset({'647'}), frozenset({'1966'}), frozenset({'1698'}), frozenset({'379'}), frozenset({'1046'}), frozenset({'1030'}), frozenset({'433'}), frozenset({'39', '156'}), frozenset({'1435', '48'}), frozenset({'38', '806'}), frozenset({'1144', '39'}), frozenset({'48', '425'}), frozenset({'39', '80'}), frozenset({'3321', '48'}), frozenset({'10515', '39', '48'}), frozenset({'39', '48', '413'}), frozenset({'286', '38', '39', '48'}), frozenset({'14099'}), frozenset({'15618'}), frozenset({'15578'}), frozenset({'13443'}), frozenset({'1543'}), frozenset({'136'}), frozenset({'514'}), frozenset({'10651'}), frozenset({'12929'}), frozenset({'2626'}), frozenset({'13032'}), frozenset({'169'}), frozenset({'10', '39'}), frozenset({'48', '175'}), frozenset({'264', '48'}), frozenset({'56', '38'}), frozenset({'441', '48'}), frozenset({'12951', '39'}), frozenset({'2810', '39'}), frozenset({'10515', '405'}), frozenset({'39', '806'}), frozenset({'48', '267'}), frozenset({'1239', '48'}), frozenset({'396', '48'}), frozenset({'39', '1986'}), frozenset({'39', '12943'}), frozenset({'39', '9617'}), frozenset({'150', '48'}), frozenset({'39', '186'}), frozenset({'39', '2241'}), frozenset({'39', '1693'}), frozenset({'48', '675'}), frozenset({'13041', '413'}), frozenset({'13334', '39', '48'}), frozenset({'39', '258', '48'}), frozenset({'39', '5152', '48'}), frozenset({'14098', '38', '39'}), frozenset({'14098', '39', '32'}), frozenset({'38', '48', '413'}), frozenset({'39', '13041', '89'}), frozenset({'32', '39', '13041'})

In [5]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#=======================分区：16=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=16
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'379'}), frozenset({'1486'}), frozenset({'251'}), frozenset({'677'}), frozenset({'447'}), frozenset({'227'}), frozenset({'211'}), frozenset({'479'}), frozenset({'2364'}), frozenset({'10875'}), frozenset({'110'}), frozenset({'136'}), frozenset({'509'}), frozenset({'2626'}), frozenset({'10605'}), frozenset({'2706'}), frozenset({'2468'}), frozenset({'170'}), frozenset({'1030'}), frozenset({'681'}), frozenset({'201'}), frozenset({'2505'}), frozenset({'338'}), frozenset({'548'}), frozenset({'2046'}), frozenset({'412'}), frozenset({'1198'}), frozenset({'589'}), frozenset({'766'}), frozenset({'789'}), frozenset({'427'}), frozenset({'9203'}), frozenset({'178'}), frozenset({'76'}), frozenset({'395', '48'}), frozenset({'237', '32'}), frozenset({'38', '48'}), frozenset({'39', '237'}), frozenset({'48', '89'}), frozenset({'48', '19'}), frozenset({'39', '1004'}), frozenset({'438', '41'}), frozenset({'10446', '48'}), frozenset({'48', '10753'}), frozenset({'161', '39'}), frozenset({'10', '39'}), frozenset({'1344', '41'}), frozenset({'39', '3270'}), frozenset({'371', '41'}), frozenset({'39', '2241'}), frozenset({'341', '39'}), frozenset({'10731', '39'}), frozenset({'48', '78'}), frozenset({'23', '39'}), frozenset({'38', '225'}), frozenset({'31', '39'}), frozenset({'1355', '48'}), frozenset({'48', '175'}), frozenset({'48', '4524'}), frozenset({'39', '5405'}), frozenset({'38', '1146'}), frozenset({'39', '48'}), frozenset({'32', '48'}), frozenset({'38', '237'}), frozenset({'39', '1986'}), frozenset({'179', '48'}), frozenset({'2958', '48'}), frozenset({'32', '225'}), frozenset({'264', '48'}), frozenset({'185', '48'}), frozenset({'39', '488'}), frozenset({'39', '225'}), frozenset({'1327', '48'}), frozenset({'39', '1146'}), frozenset({'101', '48'}), frozenset({'271', '41'}), frozenset({'39', '4070'}), frozenset({'209', '39'}), frozenset({'39', '806'}), frozenset({'39', '186'}), frozenset({'740', '39'}), frozenset({'107', '39'}), frozenset({'808', '48'}), frozenset({'39', '49'}), frozenset({'310', '39', '48'}), frozenset({'286', '38', '39'}), frozenset({'39', '2238', '48'}), frozenset({'301', '39', '48'}), frozenset({'310', '32', '48'}), frozenset({'592', '39', '48'}), frozenset({'41', '48', '76'}), frozenset({'39', '258', '48'}), frozenset({'475', '39', '48'}), frozenset({'39', '4698', '48'}), frozenset({'270', '39', '41'}), frozenset({'110', '48', '41'}), frozenset({'39', '48', '413'}), frozenset({'548', '48', '41'}), frozenset({'170', '48', '41'}), frozenset({'1578', '39', '48'}), frozenset({'10611', '39', '41'}), frozenset({'270', '271', '48'}), frozenset({'270', '39', '48', '41'}), frozenset({'10611', '39', '48', '41'}), frozenset({'38', '39', '170', '41'}), frozenset({'38', '39', '110', '41'}), frozenset({'2492'}), frozenset({'48', '749'}), frozenset({'32', '1146'}), frozenset({'2168', '41'}), frozenset({'612', '39'}), frozenset({'1859', '48', '41'}), frozenset({'533', '39', '41'}), frozenset({'39', '41', '117'}), frozenset({'855', '39', '48'}), frozenset({'475', '32', '48'}), frozenset({'1859'}), frozenset({'1481'}), frozenset({'647'}), frozenset({'1046'}), frozenset({'514'}), frozenset({'1966'}), frozenset({'15'}), frozenset({'433'}), frozenset({'39', '156'}), frozenset({'39', '1121'}), frozenset({'1783', '39'}), frozenset({'522', '39'}), frozenset({'2056', '39'}), frozenset({'32', '156'}), frozenset({'200', '48'}), frozenset({'2056', '32'}), frozenset({'38', '156'}), frozenset({'189', '48'}), frozenset({'39', '7429'}), frozenset({'48', '769'}), frozenset({'39', '1034', '769'}), frozenset({'310', '38', '48'}), frozenset({'39', '38', '1034'}), frozenset({'1198', '48', '41'}), frozenset({'38', '1034', '769'}), frozenset({'10170'}), frozenset({'1543'}), frozenset({'9579'}), frozenset({'374'}), f

In [6]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#======================w=3=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




In [7]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#======================w=4=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'348'}), frozenset({'840'}), frozenset({'1013'}), frozenset({'2413'}), frozenset({'251'}), frozenset({'381'}), frozenset({'1096'}), frozenset({'677'}), frozenset({'703'}), frozenset({'1859'}), frozenset({'170'}), frozenset({'201'}), frozenset({'374'}), frozenset({'479'}), frozenset({'681'}), frozenset({'1966'}), frozenset({'2505'}), frozenset({'3840'}), frozenset({'4198'}), frozenset({'110'}), frozenset({'338'}), frozenset({'548'}), frozenset({'1198'}), frozenset({'2046'}), frozenset({'2343'}), frozenset({'589'}), frozenset({'766'}), frozenset({'789'}), frozenset({'1233'}), frozenset({'2492'}), frozenset({'3966'}), frozenset({'76'}), frozenset({'39', '48'}), frozenset({'38', '48'}), frozenset({'39', '94'}), frozenset({'101', '48'}), frozenset({'48', '89'}), frozenset({'32', '48'}), frozenset({'179', '48'}), frozenset({'48', '189'}), frozenset({'38', '237'}), frozenset({'39', '237'}), frozenset({'39', '225'}), frozenset({'38', '225'}), frozenset({'161', '39'}), frozenset({'48', '352'}), frozenset({'237', '32'}), frozenset({'23', '39'}), frozenset({'19', '48'}), frozenset({'39', '740'}), frozenset({'522', '39'}), frozenset({'39', '1121'}), frozenset({'39', '1146'}), frozenset({'32', '1146'}), frozenset({'271', '41'}), frozenset({'1327', '48'}), frozenset({'39', '1004'}), frozenset({'48', '749'}), frozenset({'438', '41'}), frozenset({'1344', '41'}), frozenset({'185', '48'}), frozenset({'32', '1715'}), frozenset({'39', '1715'}), frozenset({'32', '225'}), frozenset({'39', '3270'}), frozenset({'2958', '48'}), frozenset({'48', '2990'}), frozenset({'286', '38', '39'}), frozenset({'475', '39', '48'}), frozenset({'270', '39', '41'}), frozenset({'170', '48', '41'}), frozenset({'592', '39', '48'}), frozenset({'110', '48', '41'}), frozenset({'39', '41', '533'}), frozenset({'475', '32', '48'}), frozenset({'310', '39', '48'}), frozenset({'301', '39', '48'}), frozenset({'310', '32', '48'}), frozenset({'1859', '48', '41'}), frozenset({'855', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'38', '39', '110', '41'}), frozenset({'38', '39', '170', '41'}), frozenset({'10170'}), frozenset({'379'}), frozenset({'13032'}), frozenset({'3294'}), frozenset({'9579'}), frozenset({'12929'}), frozenset({'14099'}), frozenset({'1046'}), frozenset({'136'}), frozenset({'4424'}), frozenset({'1313'}), frozenset({'2150'}), frozenset({'39', '49'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'48', '78'}), frozenset({'39', '186'}), frozenset({'31', '39'}), frozenset({'56', '38'}), frozenset({'264', '48'}), frozenset({'39', '156'}), frozenset({'10', '39'}), frozenset({'48', '425'}), frozenset({'39', '258', '48'}), frozenset({'39', '48', '809'}), frozenset({'270', '271', '48'}), frozenset({'39', '48', '413'}), frozenset({'2706'}), frozenset({'6343'}), frozenset({'1543'}), frozenset({'13443'}), frozenset({'15033'}), frozenset({'1481'}), frozenset({'3149'}), frozenset({'3904'}), frozenset({'14855'}), frozenset({'10605'}), frozenset({'1486'}), frozenset({'2468'}), frozenset({'1030'}), frozenset({'211'}), frozenset({'2364'}), frozenset({'412'}), frozenset({'427'}), frozenset({'9203'}), frozenset({'178'}), frozenset({'395', '48'}), frozenset({'48', '497'}), frozenset({'10446', '48'}), frozenset({'39', '2241'}), frozenset({'39', '488'}), frozenset({'48', '175'}), frozenset({'39', '10442'}), frozenset({'39', '806'}), frozenset({'48', '4524'}), frozenset({'107', '39'}), frozenset({'808', '48'}), frozenset({'249', '41'}), frozenset({'270', '39', '48', '41'}), frozenset({'15578'}), frozenset({'10515', '405'}), frozenset({'48', '267'}), frozenset({'1435', '48'}), frozenset({'150', '48'}), frozenset({'39', '80'}), frozenset({'1239', '48'}), frozenset({'396', '48'}), frozenset({'39', '1986'}), frozense

In [8]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#======================w=5=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'10605'}), frozenset({'379'}), frozenset({'251'}), frozenset({'677'}), frozenset({'643'}), frozenset({'703'}), frozenset({'170'}), frozenset({'374'}), frozenset({'479'}), frozenset({'201'}), frozenset({'681'}), frozenset({'110'}), frozenset({'338'}), frozenset({'2046'}), frozenset({'412'}), frozenset({'53'}), frozenset({'548'}), frozenset({'1198'}), frozenset({'589'}), frozenset({'789'}), frozenset({'766'}), frozenset({'136'}), frozenset({'1046'}), frozenset({'433'}), frozenset({'2492'}), frozenset({'1313'}), frozenset({'650'}), frozenset({'178'}), frozenset({'76'}), frozenset({'38', '389'}), frozenset({'39', '389'}), frozenset({'39', '48'}), frozenset({'389', '32'}), frozenset({'264', '48'}), frozenset({'10446', '48'}), frozenset({'38', '48'}), frozenset({'39', '156'}), frozenset({'32', '48'}), frozenset({'39', '12491'}), frozenset({'12473', '48'}), frozenset({'48', '425'}), frozenset({'39', '237'}), frozenset({'39', '3270'}), frozenset({'179', '48'}), frozenset({'1327', '48'}), frozenset({'39', '740'}), frozenset({'48', '78'}), frozenset({'39', '49'}), frozenset({'185', '48'}), frozenset({'23', '39'}), frozenset({'48', '12503'}), frozenset({'48', '19'}), frozenset({'48', '89'}), frozenset({'107', '39'}), frozenset({'101', '48'}), frozenset({'39', '225'}), frozenset({'56', '38'}), frozenset({'56', '39'}), frozenset({'39', '1146'}), frozenset({'39', '1004'}), frozenset({'32', '225'}), frozenset({'161', '39'}), frozenset({'2958', '48'}), frozenset({'38', '237'}), frozenset({'38', '225'}), frozenset({'31', '39'}), frozenset({'237', '32'}), frozenset({'522', '39'}), frozenset({'39', '2241'}), frozenset({'1404', '48'}), frozenset({'39', '258', '48'}), frozenset({'310', '39', '48'}), frozenset({'592', '39', '48'}), frozenset({'39', '48', '413'}), frozenset({'10515', '39', '48'}), frozenset({'39', '1578', '48'}), frozenset({'39', '2238', '48'}), frozenset({'286', '38', '39'}), frozenset({'475', '39', '48'}), frozenset({'270', '271', '48'}), frozenset({'39', '48', '649'}), frozenset({'301', '39', '48'}), frozenset({'286', '39', '38', '48'}), frozenset({'2097'}), frozenset({'1486'}), frozenset({'652'}), frozenset({'13443'}), frozenset({'3539'}), frozenset({'1859'}), frozenset({'1966'}), frozenset({'647'}), frozenset({'9579'}), frozenset({'2259'}), frozenset({'514'}), frozenset({'739'}), frozenset({'38', '1146'}), frozenset({'39', '1121'}), frozenset({'39', '1986'}), frozenset({'10', '39'}), frozenset({'438', '41'}), frozenset({'396', '48'}), frozenset({'32', '1146'}), frozenset({'271', '41'}), frozenset({'2056', '39'}), frozenset({'1344', '41'}), frozenset({'123', '41'}), frozenset({'48', '769'}), frozenset({'48', '675'}), frozenset({'1593', '48'}), frozenset({'48', '175'}), frozenset({'200', '48'}), frozenset({'39', '9617'}), frozenset({'855', '39', '48'}), frozenset({'170', '48', '41'}), frozenset({'110', '48', '41'}), frozenset({'39', '533', '41'}), frozenset({'270', '39', '41'}), frozenset({'39', '117', '41'}), frozenset({'39', '1034', '769'}), frozenset({'39', '38', '170', '41'}), frozenset({'39', '38', '110', '41'}), frozenset({'270', '39', '48', '41'}), frozenset({'15578'}), frozenset({'10515', '405'}), frozenset({'39', '806'}), frozenset({'48', '267'}), frozenset({'1435', '48'}), frozenset({'150', '48'}), frozenset({'39', '80'}), frozenset({'1239', '48'}), frozenset({'39', '186'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'39', '12943'}), frozenset({'12951', '39'}), frozenset({'12925', '41'}), frozenset({'13041', '413'}), frozenset({'41', '15832'}), frozenset({'65', '15832'}), frozenset({'10423', '48'}), frozenset({'13334', '39', '48'}), frozenset({'39', '5152', '48'}), frozenset({'32', '48', '413'}), frozenset({'38', '48', '413'}), frozenset({'39', '48'

In [9]:
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'\
#======================w=2=====================
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools


# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'

start = time.clock()
t0 = time.time()


def getindex(index, it):
  for t in it:
    yield (t,index)

def splitlines(x):
    y = ' '.join(x.split())
    return y
  
def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
                
def ProduceSubset(cset,cur_round):
    N=len(cset)
    for i in range(2**N):
        combo = []
        for j in range(N):
            #test jth bit of integer i
            if(i >> j ) % 2 == 1:
                combo.append(cset[j])
        yield frozenset(combo) 
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
       # combo.append(csets)
        #combo.append(flag)
        #combo.append(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)
            
#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区
#display(dbutils.fs.ls("/FileStore/tables"))
spl_num=8
input_rdd = sc.textFile('/FileStore/tables/retail.txt',spl_num)
#print(input_rdd.collect())

input_rdd2 = input_rdd.mapPartitionsWithIndex(getindex)
#print('in\n:',input_rdd.collect())
input_rdd2.cache()

frqtemp=[]
for irdd in range(0,spl_num):
  rdd_i = input_rdd2.filter(lambda kv: kv[1]==irdd).map(lambda x:x[0])
  #print('rddi:\n',rdd_i.collect())
  splitedline = rdd_i.map(splitlines)
  transations = splitedline.map(lambda x: x.split(' '))
  transations.collect()
  transations.cache()

  numRecords = float(transations.count())
  #print ("numRecords：" + str(numRecords))
  freqThre = numRecords * support
  #print ("frequency threshold：" + str(freqThre))
        
  #计算频繁1项集

  candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
  FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
  .filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
  #print ('cur_freqset_1',FreqSet_1.collect())
  preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
  #print (preFreSets.collect())
  frqtemp.append(preFreSets.map(lambda x: frozenset([x])).collect())
            
  for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    #print (preFreSets.collect())
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        #print ('elements'+str(elements))
        #print(preFreSets.map(lambda x: set(x)).collect())
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
       # print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
        #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
       # print ('elemen'+str(elements))
        #print(preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements)).collect())
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        #print ('canSets'+str(canSets.collect())+'\n')
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        #print ('cantemp\n'+str(cantemp)+'\n')
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        #print ('candi\n'+str(candidates.collect())+'\n')
        candidates = candidates.map(lambda x: frozenset(x))
       #candidates = generateCandidates(pre, r, elements,canSets)
        #print ('candi2\n'+str(candidates.collect())+'\n')
    
    
    #print candidates.collect()
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    
    
    
    
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .filter(lambda x :x[1] >= freqThre)

    
    preFreSets = curFreqSet.map(lambda x :x[0])
    #print ('curFreqSets in round'+str(r)+':\n'+str(curFreqSet.collect())+'\n')
    #print (preFreSets.collect())
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append(preFreSets.collect())
#print(irdd,'frqtemp:\n',frqtemp)


"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates_list = list(itertools.chain.from_iterable(frqtemp))
global_candidates = sc.parallelize(global_candidates_list,8).distinct()
print('global:\n',global_candidates.collect())
"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
transationss = input_rdd.map(splitlines).map(lambda x: x.split(' '))
transationss.collect()
transationss.cache()
original_data=sc.broadcast(transationss.collect())
numRecords = float(transationss.count())
#print('ori:\n',original_data.value)    

"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates\
.map(lambda x:(x,len([transac for transac in original_data.value if x.issubset(frozenset(transac))])/len(original_data.value) ))\
    .filter(lambda x: x[1]>=support).collectAsMap()

print ('global_frequent_itemsets:\n',global_frequent_itemsets)



   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from six.moves import range
global:
 [frozenset({'348'}), frozenset({'840'}), frozenset({'1013'}), frozenset({'2413'}), frozenset({'251'}), frozenset({'381'}), frozenset({'1096'}), frozenset({'677'}), frozenset({'703'}), frozenset({'1859'}), frozenset({'170'}), frozenset({'201'}), frozenset({'374'}), frozenset({'479'}), frozenset({'681'}), frozenset({'1966'}), frozenset({'2505'}), frozenset({'3840'}), frozenset({'4198'}), frozenset({'110'}), frozenset({'338'}), frozenset({'548'}), frozenset({'1198'}), frozenset({'2046'}), frozenset({'2343'}), frozenset({'589'}), frozenset({'766'}), frozenset({'789'}), frozenset({'1233'}), frozenset({'2492'}), frozenset({'3966'}), frozenset({'76'}), frozenset({'39', '48'}), frozenset({'38', '48'}), frozenset({'39', '94'}), frozenset({'101', '48'}), frozenset({'48', '89'}), frozenset({'32', '48'}), frozenset({'179', '48'}), frozenset({'48', '189'}), frozenset({'38', '237'}), frozenset({'39', '237'}), frozenset({'39', '225'}), frozenset({'38', '225'}), frozenset({'161', '39'}), frozenset({'48', '352'}), frozenset({'237', '32'}), frozenset({'23', '39'}), frozenset({'19', '48'}), frozenset({'39', '740'}), frozenset({'522', '39'}), frozenset({'39', '1121'}), frozenset({'39', '1146'}), frozenset({'32', '1146'}), frozenset({'271', '41'}), frozenset({'1327', '48'}), frozenset({'39', '1004'}), frozenset({'48', '749'}), frozenset({'438', '41'}), frozenset({'1344', '41'}), frozenset({'185', '48'}), frozenset({'32', '1715'}), frozenset({'39', '1715'}), frozenset({'32', '225'}), frozenset({'39', '3270'}), frozenset({'2958', '48'}), frozenset({'48', '2990'}), frozenset({'286', '38', '39'}), frozenset({'475', '39', '48'}), frozenset({'270', '39', '41'}), frozenset({'170', '48', '41'}), frozenset({'592', '39', '48'}), frozenset({'110', '48', '41'}), frozenset({'533', '39', '41'}), frozenset({'475', '32', '48'}), frozenset({'310', '39', '48'}), frozenset({'301', '39', '48'}), frozenset({'310', '32', '48'}), frozenset({'1859', '48', '41'}), frozenset({'855', '39', '48'}), frozenset({'39', '2238', '48'}), frozenset({'38', '39', '110', '41'}), frozenset({'38', '39', '170', '41'}), frozenset({'10170'}), frozenset({'379'}), frozenset({'10605'}), frozenset({'1486'}), frozenset({'2468'}), frozenset({'1030'}), frozenset({'211'}), frozenset({'2364'}), frozenset({'412'}), frozenset({'136'}), frozenset({'427'}), frozenset({'9203'}), frozenset({'178'}), frozenset({'395', '48'}), frozenset({'48', '497'}), frozenset({'10446', '48'}), frozenset({'10', '39'}), frozenset({'264', '48'}), frozenset({'56', '38'}), frozenset({'39', '2241'}), frozenset({'39', '488'}), frozenset({'48', '78'}), frozenset({'31', '39'}), frozenset({'48', '175'}), frozenset({'39', '10442'}), frozenset({'39', '806'}), frozenset({'48', '4524'}), frozenset({'107', '39'}), frozenset({'808', '48'}), frozenset({'249', '41'}), frozenset({'39', '49'}), frozenset({'39', '258', '48'}), frozenset({'39', '48', '413'}), frozenset({'270', '39', '48', '41'}), frozenset({'13032'}), frozenset({'3294'}), frozenset({'9579'}), frozenset({'12929'}), frozenset({'14099'}), frozenset({'1046'}), frozenset({'4424'}), frozenset({'1313'}), frozenset({'2150'}), frozenset({'39', '1693'}), frozenset({'441', '48'}), frozenset({'39', '186'}), frozenset({'39', '156'}), frozenset({'48', '425'}), frozenset({'39', '48', '809'}), frozenset({'270', '271', '48'}), frozenset({'2706'}), frozenset({'6343'}), frozenset({'1543'}), frozenset({'13443'}), frozenset({'15033'}), frozenset({'1481'}), frozenset({'3149'}), frozenset({'3904'}), frozenset({'14855'}), frozenset({'15578'}), frozenset({'10515', '405'}), frozenset({'48', '267'}), frozenset({'1435', '48'}), frozenset({'150', '48'}), frozenset({'39', '80'}), frozenset({'1239', '48'}), frozenset({'396', '48'}), frozenset({'39', '1986'}), frozense